In [1]:
import json
import re
import datetime
import requests
import os

flux_project = 'Flux.WST.Inventory.API'
f = open('WST.Inventory.API-swagger.json')
data = json.load(f)

In [4]:
dirs = os.listdir(f'C:\\Users\\adangelo\\code\\{flux_project}\\{flux_project}\\Interfaces')
interfaces = []
test_files = os.listdir(f'C:\\Users\\adangelo\\code\\{flux_project}\\{flux_project}\\TestScripts')
flux_api_methods = []

for file in dirs:
  text_file = open(f'C:\\Users\\adangelo\\code\\{flux_project}\\{flux_project}\\Interfaces\\' + file, 'r')
  lines = text_file.readlines()

  for i, line in enumerate(lines):
    if re.search('\[Post|\[Get|\[Put|\[Patch|\[Delete', line):
      interfaces.append(line)
      method = re.search('\w*\(', lines[i+1]).group(0)[:-1]
      flux_api_methods.append(method)

print(interfaces)
print(flux_api_methods)

['        [Patch("/api/v1/facilities/{facilityCode}/product-type/{productType}/dereserve/{numberOfDays}")]\n', '        [Get("/api/v1/{facilityCode}/batches")]\n', '        [Get("/api/v1/{facilityCode}/batches/{batchId}/units")]\n', '        [Get("/api/v1/device/virtualidentifiers/facilities/{facility}")]\n', '        [Get("/api/v1/IssueLocations/facilities/{facility}")]\n', '        [Get("/api/v1/units/facilities/{facility}/units/{donationId}/productcodes/{productCode}/antibodies")]\n', '        [Get("/api/v1/units/facilities/{facility}/units/{donationId}/productcodes/{productCode}/antigens")]\n', '        [Get("/api/v1/units/facilities/{facility}/units/{donationId}/productcodes/{productCode}/attributes")]\n', '        [Get("/api/v1/units/facilities/{facility}/units/{donationId}/productcodes/{productCode}")]\n', '        [Put("/api/v1/device/virtualIdentifiers/facilities/{facility}/devices/{device}/identifiers/{identifier}")]\n']
['DereservationAsync', 'GetBatchesAsync', 'GetBatchUnit

In [5]:
def get_test_file_paths_on_flux_project():
    test_file_paths = []
    test_files_root = f'C:\\Users\\adangelo\\code\\{flux_project}\\{flux_project}\\TestScripts'
    for root, test_subdirs, files in os.walk(test_files_root):
        for file in files:
            test_file_paths.append(os.path.join(root, file))
    return test_file_paths

test_file_paths = get_test_file_paths_on_flux_project()
test_file_paths

['C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Dereservation\\DereservationTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Device\\DeviceVirtualIdentifierTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Electronic\\GetElectronicBatchesTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Electronic\\GetElectronicBatchUnitsTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\IssueLocations\\IssueLocationsTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Unit\\GetUnitAntibodiesTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Unit\\GetUnitAntigensTests.cs',
 'C:\\Users\\adangelo\\code\\Flux.WST.Inventory.API\\Flux.WST.Inventory.API\\TestScripts\\Uni

In [6]:

def count_method_calls(method, test_file_paths):
  count = 0
  for file in test_file_paths:
    with open(file) as f:
      lines = f.readlines()
      for line in lines:
        if f'{method}' in line:
          count += 1
  return count

method_counts = {}
for method in flux_api_methods:
  method_counts[method] = count_method_calls(method, test_file_paths)
method_counts

{'DereservationAsync': 4,
 'GetBatchesAsync': 3,
 'GetBatchUnitsAsync': 4,
 'GetDeviceVirtualIdentifiersAsync': 4,
 'GetIssueLocationsAsync': 3,
 'GetUnitAntibodiesAsync': 3,
 'GetUnitAntigensAsync': 3,
 'GetUnitAttributesAsync': 3,
 'GetUnitAsync': 6,
 'PutDeviceVirtualIdentifiersAsync': 3}

In [7]:
def get_endpoints_calls_from_flux_project(interfaces):
  flux_operations = []
  flux_paths = []

  for interface in interfaces:
    flux_operations.append(re.search('\[Get|\[Post|\[Patch|\[Put', interface).group(0)[1:].lower())
    flux_paths.append(re.search('(?<=\").*(?=\")', interface).group(0).lower())

  return flux_paths, flux_operations

endpoints_on_flux_project, endpoint_operations_on_flux_project = get_endpoints_calls_from_flux_project(interfaces)
print(endpoints_on_flux_project)
print(endpoint_operations_on_flux_project)
flux_api_calls = {}

for i, endpoint in enumerate(endpoints_on_flux_project):
  flux_api_calls[endpoint] = { 
    'operation': endpoint_operations_on_flux_project[i],
    'method': flux_api_methods[i],
    'count': method_counts[flux_api_methods[i]],
  }

flux_api_calls

['/api/v1/facilities/{facilitycode}/product-type/{producttype}/dereserve/{numberofdays}', '/api/v1/{facilitycode}/batches', '/api/v1/{facilitycode}/batches/{batchid}/units', '/api/v1/device/virtualidentifiers/facilities/{facility}', '/api/v1/issuelocations/facilities/{facility}', '/api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/antibodies', '/api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/antigens', '/api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/attributes', '/api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}', '/api/v1/device/virtualidentifiers/facilities/{facility}/devices/{device}/identifiers/{identifier}']
['patch', 'get', 'get', 'get', 'get', 'get', 'get', 'get', 'get', 'put']


{'/api/v1/facilities/{facilitycode}/product-type/{producttype}/dereserve/{numberofdays}': {'operation': 'patch',
  'method': 'DereservationAsync',
  'count': 4},
 '/api/v1/{facilitycode}/batches': {'operation': 'get',
  'method': 'GetBatchesAsync',
  'count': 3},
 '/api/v1/{facilitycode}/batches/{batchid}/units': {'operation': 'get',
  'method': 'GetBatchUnitsAsync',
  'count': 4},
 '/api/v1/device/virtualidentifiers/facilities/{facility}': {'operation': 'get',
  'method': 'GetDeviceVirtualIdentifiersAsync',
  'count': 4},
 '/api/v1/issuelocations/facilities/{facility}': {'operation': 'get',
  'method': 'GetIssueLocationsAsync',
  'count': 3},
 '/api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/antibodies': {'operation': 'get',
  'method': 'GetUnitAntibodiesAsync',
  'count': 3},
 '/api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/antigens': {'operation': 'get',
  'method': 'GetUnitAntigensAsync',
  'count': 3},
 '/api/v1/

In [56]:
def get_api_endpoints_and_operations_from_swagger_data(swagger_data):
  api_endpoints = []
  api_operations = []

  for path in data['paths']:
    api_endpoints.append(path.lower())
    api_operations.append(list(data['paths'][path].keys())[0])

  return api_endpoints, api_operations

api_endpoints, api_operations = get_api_endpoints_and_operations_from_swagger_data(data)
print(api_endpoints)
print(api_operations)

['/api/v1/bloodtrack/stockmovement/sites/{facilitycode}', '/api/v1/comment/gethardcodedcomments', '/api/v1/sites/{sitecode}/configuration/emergencyissuecompatibilitymatricies', '/api/v1/sites/{sitecode}/configuration/saveemergencyissuecompatibilitymatricies', '/api/v1/facilities/{facilitycode}/product-type/{producttype}/dereserve/{numberofdays}', '/api/v1/derivatives/facilities/{facility}/lots/{lot}/sublots/{sublot}/derivativecodes/{derivativecode}', '/api/v1/derivatives/savederivative', '/api/v1/derivatives/issueforstockmovement', '/api/v1/derivatives/updatelocationforstockmovement', '/api/v1/derivatives/issueforstocktransfer/facilities/{facility}/lots/{lot}/sublots/{sublot}/derivatives/{derivativecode}', '/api/v1/derivatives/bloodtrack/stockmovement', '/api/v1/derivatives/bloodtrack/stocktransfer', '/api/v1/derivatives/issueforreturntostock', '/api/v1/derivatives/issueforfateofunit', '/api/v1/devices/derivatives/facilities/{facilitycode}/lots/{lot}/sublots/{sublot}/removefromstorage'

In [79]:
exercised_endpoints = 0
total_of_api_endpoints = len(api_endpoints)

for endpoint in flux_api_calls:
  if flux_api_calls[endpoint]["count"] > 0:
    exercised_endpoints += 1

print(f'---------------------------------- {flux_project} ----------------------------------')
print(f'Endpoints Coverage Percentage: {round((exercised_endpoints / total_of_api_endpoints * 100), 1)}%')
print('')
print(f'Endpoints Covered At {datetime.datetime.now().strftime("%x")}:')
for endpoint in flux_api_calls:
  print(f'  {flux_api_calls[endpoint]["operation"]} {endpoint} ')
  print(f'    (exercised by {flux_api_calls[endpoint]["count"]} tests)')

---------------------------------- Flux.WST.Inventory.API ----------------------------------
Endpoints Coverage Percentage: 14.5%

Endpoints Covered At 05/19/23:
  patch /api/v1/facilities/{facilitycode}/product-type/{producttype}/dereserve/{numberofdays} 
    (exercised by 4 tests)
  get /api/v1/{facilitycode}/batches 
    (exercised by 3 tests)
  get /api/v1/{facilitycode}/batches/{batchid}/units 
    (exercised by 4 tests)
  get /api/v1/device/virtualidentifiers/facilities/{facility} 
    (exercised by 4 tests)
  get /api/v1/issuelocations/facilities/{facility} 
    (exercised by 3 tests)
  get /api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/antibodies 
    (exercised by 3 tests)
  get /api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/antigens 
    (exercised by 3 tests)
  get /api/v1/units/facilities/{facility}/units/{donationid}/productcodes/{productcode}/attributes 
    (exercised by 3 tests)
  get /api/v1/units/fa

In [2]:
f = open(f'api-urls.json')
apiurls = json.load(f)

for apiurl in apiurls:
  url = apiurl['apiSpecificationUrl']
  print(url)

https://bmtfnq22app01.wellsky.com/wst.scheduler/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.antibodytesting/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.patient/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.antigentesting/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.inventory/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.bloodtrack/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.orders/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.administration/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.interface/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.home/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.tenantprofile/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.auditing/docs/swagger.json
https://bmtfnq22app01.wellsky.com/ws.barcode/docs/swagger.json
https://bmtfnq22app01.wellsky.com/wst.processmigration/docs/swagger.json


In [4]:
resp = requests.get("bmtfnq22app01.wellsky.com/wst.home/docs/swagger.json")
data = resp.json() # Check the JSON Response Content documentation below
data.save('./temp/swagger.json')
print(data)

MissingSchema: Invalid URL 'bmtfnq22app01.wellsky.com/wst.home/docs/swagger.json': No scheme supplied. Perhaps you meant https://bmtfnq22app01.wellsky.com/wst.home/docs/swagger.json?